In [3]:
import pandas as pd

from factue.utils.vars import PROJECT_ROOT
from factue.utils.viz import disp

In [4]:
sample_out = (
    PROJECT_ROOT
    / "data/02_cleaned_reference_test"
    / "dev"
    / "dev-eng"
    / "batch_0000.parquet"
)
df = pd.read_parquet(sample_out)
disp(df[["text", "reference", "check"]])

,text,reference,check
0,"The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More None",Photo shows Louis Armstrong as a child,"{ 'questions': [ {'id': 'meaning_preserved', 'answer': 'false', 'errors': ['The claim does not mention the Karnofsky family employing Louis Armstrong as a child, but rather that they employed a 7-year-old boy who later became a professional musician, which is likely referring to Louis Armstrong.']}, {'id': 'correct_named_entities', 'answer': 'true'}, {'id': 'correct_numbers', 'answer': 'false', 'errors': ['The claim mentions the child being shown as a photo, but does not mention any specific age or number related to the Karnofsky family employing him. However, it is mentioned that he was 7 years old when employed and 12 when no longer under their employ.']}, {'id': 'no_added_data', 'answer': 'false', 'errors': ['The claim mentions a photo of Louis Armstrong as a child, which is not present in the original text.']}, {'id': 'no_missing_data', 'answer': 'true'}, {'id': 'alternative_claim_formulation', 'answer': 'true', 'errors': ['A more accurate formulation would be: ""The Karnofsky Jewish family employed a 7-year-old boy who later became Louis Armstrong.""']} ] }"
1,"Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 mill Co This little guy was rescued after his mother was murdered by a hunter. His eyes say it all, don't you think? If you want to ban trophy hunting forever, spread this everywhere. Image Credit: FB. Changes: cropped, resized, text added. https://bit.ly/3b2vwas",This leopard cub's mother was killed by a trophy hunter,"{ 'questions': [ {'id': 'meaning_preserved', 'answer': 'true'}, {'id': 'correct_named_entities', 'answer': 'false', 'errors': ['""leopard cub"" is not present in the original text']}, {'id': 'correct_numbers', 'answer': 'true'}, {'id': 'no_added_data', 'answer': 'false', 'errors': ['""rescued"", ""murdered"", ""Image Credit: FB. Changes: cropped, resized, text added."" are not present in the original text']}, {'id': 'no_missing_data', 'answer': 'true'}, {'id': 'alternative_claim_formulation', 'answer': 'true'} ] }"
2,Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert None,Videos show current situation of Hyderabad amid heavy rain,"{ 'questions': [ {'id': 'meaning_preserved', 'answer': 'false', 'errors': ['The claim does not mention the heavy rain or crocodile alert, which are present in the original text.']}, {'id': 'correct_named_entities', 'a

In [23]:
val = df["check"].iloc[0]
val

'{\n    \'questions\': [\n        {\'id\': \'meaning_preserved\', \'answer\': \'false\', \'errors\': [\'The claim does not mention the Karnofsky family employing Louis Armstrong as a child, but rather that they employed a 7-year-old boy who later became a professional musician, which is likely referring to Louis Armstrong.\']},\n        {\'id\': \'correct_named_entities\', \'answer\': \'true\'},\n        {\'id\': \'correct_numbers\', \'answer\': \'false\', \'errors\': [\'The claim mentions the child being shown as a photo, but does not mention any specific age or number related to the Karnofsky family employing him. However, it is mentioned that he was 7 years old when employed and 12 when no longer under their employ.\']},\n        {\'id\': \'no_added_data\', \'answer\': \'false\', \'errors\': [\'The claim mentions a photo of Louis Armstrong as a child, which is not present in the original text.\']},\n        {\'id\': \'no_missing_data\', \'answer\': \'true\'},\n        {\'id\': \'alt

In [26]:
ast.literal_eval(val)

{'questions': [{'id': 'meaning_preserved',
   'answer': 'false',
   'errors': ['The claim does not mention the Karnofsky family employing Louis Armstrong as a child, but rather that they employed a 7-year-old boy who later became a professional musician, which is likely referring to Louis Armstrong.']},
  {'id': 'correct_named_entities', 'answer': 'true'},
  {'id': 'correct_numbers',
   'answer': 'false',
   'errors': ['The claim mentions the child being shown as a photo, but does not mention any specific age or number related to the Karnofsky family employing him. However, it is mentioned that he was 7 years old when employed and 12 when no longer under their employ.']},
  {'id': 'no_added_data',
   'answer': 'false',
   'errors': ['The claim mentions a photo of Louis Armstrong as a child, which is not present in the original text.']},
  {'id': 'no_missing_data', 'answer': 'true'},
  {'id': 'alternative_claim_formulation',
   'answer': 'true',
   'errors': ['A more accurate formulatio

In [16]:
safe_parse_json(df["check"].iloc[6])

{'questions': [{'id': 'meaning_preserved',
   'answer': 'false',
   'errors': ['The claim does not mention that Amitabh Bachchan was hospitalised, only that he thanked healthcare workers after being hospitalised.']},
  {'id': 'correct_named_entities', 'answer': 'true'},
  {'id': 'correct_numbers', 'answer': 'true'},
  {'id': 'no_added_data',
   'answer': 'false',
   'errors': ['The claim mentions the year 2020, which is not present in the original text.']},
  {'id': 'no_missing_data',
   'answer': 'false',
   'errors': ['The claim does not mention that Amitabh Bachchan tested positive for Covid-19, which is mentioned in the post.']},
  {'id': 'alternative_claim_formulation', 'answer': 'true'}]}

In [27]:
import pandas as pd
import json
from typing import Any, Dict

import ast


def safe_json_loads(s):
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        return ast.literal_eval(s)


def expand_json_column(df, column_name):
    expanded_rows = []
    for idx, row in df.iterrows():
        json_data = safe_json_loads(row[column_name])
        questions = json_data.get("questions", [])
        record = {}
        for q in questions:
            q_id = q.get("id")
            answer = q.get("answer")
            errors = q.get("errors", [])
            record[f"{q_id}_answer"] = answer
            record[f"{q_id}_errors"] = errors
        expanded_rows.append(record)
    expanded_df = pd.DataFrame(expanded_rows)
    return pd.concat([df.reset_index(drop=True), expanded_df], axis=1)


# Expand the JSON column into new columns
df_expanded = expand_json_column(df[["text", "reference", "check"]], "check")
disp(df_expanded)

,text,reference,check,meaning_preserved_answer,meaning_preserved_errors,correct_named_entities_answer,correct_named_entities_errors,correct_numbers_answer,correct_numbers_errors,no_added_data_answer,no_added_data_errors,no_missing_data_answer,no_missing_data_errors,alternative_claim_formulation_answer,alternative_claim_formulation_errors
0,"The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More None",Photo shows Louis Armstrong as a child,"{ 'questions': [ {'id': 'meaning_preserved', 'answer': 'false', 'errors': ['The claim does not mention the Karnofsky family employing Louis Armstrong as a child, but rather that they employed a 7-year-old boy who later became a professional musician, which is likely referring to Louis Armstrong.']}, {'id': 'correct_named_entities', 'answer': 'true'}, {'id': 'correct_numbers', 'answer': 'false', 'errors': ['The claim mentions the child being shown as a photo, but does not mention any specific age or number related to the Karnofsky family employing him. However, it is mentioned that he was 7 years old when employed and 12 when no longer under their employ.']}, {'id': 'no_added_data', 'answer': 'false', 'errors': ['The claim mentions a photo of Louis Armstrong as a child, which is not present in the original text.']}, {'id': 'no_missing_data', 'answer': 'true'}, {'id': 'alternative_claim_formulation', 'answer': 'true', 'errors': ['A more accurate formulation would be: ""The Karnofsky Jewish family employed a 7-year-old boy who later became Louis Armstrong.""']} ] }",false,"['The claim does not mention the Karnofsky family employing Louis Armstrong as a child, but rather that they employed a 7-year-old boy who later became a professional musician, which is likely referring to Louis Armstrong.']",true,[],false,"['The claim mentions the child being shown as a photo, but does not mention any specific age or number related to the Karnofsky family employing him. However, it is mentioned that he was 7 years old when employed and 12 when no longer under their employ.']",false,"['The claim mentions a photo of Louis Armstrong as a child, which is not present in the original text.']",true,[],true,"['A more accurate formulation would be: ""The Karnofsky Jewish family employed a 7-year-old boy who later became Louis Armstrong.""']"
1,"Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 mill Co This little guy was rescued after his mother was murdered by a hunter. His eyes say it all, don't you think? If you want to ban trophy hunting forever, spread this everywhere. Image Credit: FB. Changes: crop